In [1]:
%pip install umap-learn
%pip install kmapper
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import manifold
import umap
from kmapper import KeplerMapper
from sklearn.preprocessing import MinMaxScaler
import kmapper as km
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [18]:
# Cargar los datos desde el archivo CSV
df = pd.read_csv('Microdatos_Aspirantes_y_Admitidos_en_la_UNAL_20240311.csv')

df = df[(df['NIVEL'] == 'Pregrado') & (df['PTOTAL'] > 10) & (df['YEAR'] == 2023)]

# Columnas a eliminar del DataFrame
columnas_a_eliminar = ['RANGO_ANO_TERMINACION', 'MODALIDAD', 'AREA_CINE',
                       'COD_CIU_RES', 'COD_DEP_NAC', 'CD_CINE', 'SEMESTRE',
                       'CA_CINE', 'AREAC_SNIES', 'PROGRAMA', 'SNIES_PROGRA',
                       'INS_SEDE_NOMBRE', 'SNIES_SEDE', 'PEAMA', 'PAES',
                       'TIPO_INS', 'MOD_INS', 'TIPO_DISC', 'CODS_NAC',
                       'NACIONALIDAD', 'LAT_CIU_RES', 'LON_CIU_RES',
                       'TIPO_NIVEL', 'DEP_NAC', 'CIU_NAC', 'COD_CIU_NAC',
                       'LAT_CIU_NAC', 'LON_CIU_NAC', 'CIU_RES', 'DEP_RES',
                       'COD_DEP_RES', 'YEAR', 'CODN_NAC', 'NIVEL']

# Eliminar las columnas especificadas
data = df.drop(columnas_a_eliminar, axis=1)

# Definir el mapeo de valores para 'ESTRATO'
mapeo_estrato = {'ND/NE': 0, 'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3, 'Estrato 4': 4,
                  'Estrato 5': 5, 'Estrato 6': 6}
# Aplicar el mapeo a la columna 'ESTRATO'
data['ESTRATO'] = data['ESTRATO'].replace(mapeo_estrato)

# Calcular la distribución de los estratos existentes en los datos
distribucion_estratos = data['ESTRATO'].value_counts(normalize=True)

# Remover el estrato '0' de la distribución
distribucion_estratos_sin_cero = distribucion_estratos.drop(0)

# Normalizar las probabilidades para asegurarse de que sumen 1
probabilidades_normalizadas = distribucion_estratos_sin_cero / distribucion_estratos_sin_cero.sum()

# Elegir aleatoriamente un estrato basado en la distribución existente
nuevo_estrato = np.random.choice(probabilidades_normalizadas.index, p=probabilidades_normalizadas.values)

# Reemplazar los valores de '0' en la columna 'ESTRATO' con el nuevo estrato seleccionado
data.loc[data['ESTRATO'] == 0, 'ESTRATO'] = nuevo_estrato

# # Definir el mapeo de valores para SEXO
mapeo_sexo = {'Mujeres': 0, 'Hombres': 1, 'Transgénero': 2}

# # Aplicar el mapeo a la columna 'SEXO'
data['SEXO'] = data['SEXO'].replace(mapeo_sexo)

# Definir el mapeo de valores para 'DISCAPACIDAD'
mapeo_discapacidad = {'Sí': 0, 'No': 1}
# Aplicar el mapeo a la columna 'DISCAPACIDAD'
data['DISCAPACIDAD'] = data['DISCAPACIDAD'].replace(mapeo_discapacidad)

mapeo_admitido = {'Sí': 0, 'No': 1}
# Aplicar el mapeo a la columna 'ADMITIDO'
data['ADMITIDO'] = data['ADMITIDO'].replace(mapeo_admitido)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80546 entries, 400435 to 493794
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   EDAD          80546 non-null  int64  
 1   SEXO          80546 non-null  int64  
 2   ESTRATO       80546 non-null  int64  
 3   DISCAPACIDAD  80546 non-null  int64  
 4   PTOTAL        80546 non-null  float64
 5   ADMITIDO      80546 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 4.3 MB


In [19]:
#Muestrear los datos
#data = data.sample(n=100000, random_state=42)

# Convertir DataFrame a un arreglo de numpy
data_array = data.values
y = data_array[:, 5].astype(float)
z = data_array[:, 1].astype(float)
w = data_array[:, 2].astype(float)
# Inicialización de KeplerMapper
mapper = km.KeplerMapper()

# Ajuste y transformación de datos
projected_data = mapper.fit_transform(data_array,
                                      projection= [manifold.Isomap(n_components=3, n_jobs=-1),
                                                   umap.UMAP(n_components=2, random_state=1)],
                                      scaler=[ MinMaxScaler(), MinMaxScaler()])
#[manifold.Isomap(n_components=3, n_jobs=-1),
#  [None, MinMaxScaler()]   umap.UMAP(n_components=2, random_state=1)]
# Creación de un cubrimiento (cover)
cover = km.Cover(n_cubes=15, perc_overlap=0.55)  # Aumentar el número de cubos para cubrir mejor los datos

# Mapeo de los datos con una distancia más grande
graph = mapper.map(projected_data, data_array, cover=cover, clusterer=km.cluster.DBSCAN(eps=10, min_samples=70))
color_values = np.c_[y, z, w]
columna_combinada = y * 100 + z * 10 + w
mapper.visualize(
    graph,
    path_html="universidad_nacional_output2023.html",
    title="Mapeo de datos de la Universidad Nacional",
    custom_tooltips=columna_combinada,
    color_values=color_values,
    color_function_name= ["ADMITIDO","SEXO", "ESTRATO"],
    node_color_function=["mean", "std", "median", "max" ]
)
# html = mapper.visualize(graph, color_values=color_values, path_html="universidad_nacional_output.html",
#                         title="Mapeo de datos de la Universidad Nacional ")

print("Visualización generada con éxito.")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\manifold\_isomap.py:383: UserWarning: The number of connected components of the neighbors graph is 38 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
c:\ProgramData\anaconda3\Lib\site-packages\scipy\sparse\_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
c:\ProgramData\anaconda3\Lib\site-packages\scipy\sparse\_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
c:\ProgramData\anaconda3\Lib\site-packages\scipy\sparse\_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
c:\ProgramData\anaconda3\L

Visualización generada con éxito.
